## Analysis of headlines #1

In this task we are going to use the Paraphraser dataset to do basic text analysis operations on news headlines.
The dataset is a large .json file.



### Getting the dataset

In [18]:
from os import remove
from shutil import copyfileobj
import zipfile
import requests

def download_file(url):
    compressed_dataset = 'Headlines.zip'
    with requests.get(url, stream=True) as r:
        with open(compressed_dataset, 'wb') as f:
            copyfileobj(r.raw, f)
            
    archive = zipfile.ZipFile(compressed_dataset)
    for file in archive.namelist():
        if file.endswith('.json'):
            archive.extract(file)
            archive.close() # close file
            os.remove(compressed_dataset) 
    return file

In [19]:
# download_file('http://paraphraser.ru/download/get?file_id=7')

### Getting familiar with data

In [ ]:
import json
with open("ParaPhraserPlus/ParaPhraserPlus.json") as data_set:
    HEADLINE_SET = json.load(data_set)

In [ ]:
HEADLINE_SET['0'].keys()

In [ ]:
list(HEADLINE_SET.items())[0]

In [ ]:
list(HEADLINE_SET.items())[0][1]['headlines']

In [ ]:
len(HEADLINE_SET)

### Starting the stage of analysis

In [ ]:
import json
import nltk, razdel
from nltk.corpus import stopwords
from pymystem3 import Mystem
import string
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Download nltk packages used in this example
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

In [ ]:
# Customize list of stopwords as needed. Here, we add common
# punctuation and contraction artifacts.
punctuations = list(string.punctuation)
stop_words = stopwords.words('russian') + punctuations

In [ ]:
def basic_preprocess_headlines(article_id):
    sentences = []
    tokens = []
    
    rubric = list(HEADLINE_SET.items())[article_id][1]['rubric']
    headlines = list(HEADLINE_SET.items())[article_id][1]['headlines']
    
    sentences += [_.text for each_headline in headlines for _ in razdel.sentenize(each_headline)]
    tokens += [token.text.lower() for sentence in sentences 
               for token in razdel.tokenize(sentence)]
        
    corpus = [word for word in tokens if word[0] not in stop_words]
    fdist = nltk.FreqDist(corpus)
    return (article_id, rubric, sentences, tokens, corpus, fdist)

In [ ]:
article_id, rubric, sentences, tokens, corpus, fdist = basic_preprocess_headlines(25)

In [ ]:
%matplotlib inline
import warnings
from IPython.display import display
warnings.filterwarnings("ignore")

mystem = Mystem()
#Preprocess function
def informative_preprocess_headline(sentence):
    lemmas = mystem.lemmatize(sentence.lower())
    sentence_info = mystem.analyze(sentence)
    analysis_info = list(filter(lambda x: 'analysis' in x, sentence_info))
    
    lemmas = [lemma for lemma in lemmas if lemma not in stop_words\
              and lemma != " " \
              and lemma.strip() not in punctuations]

    sentence = " ".join(lemmas)
    processed_sentence = [word for word in sentence.split(' ')]

    return (processed_sentence, analysis_info)

    # Basic stats
def headline_stats():
    num_words = sum([i for i in fdist.values()])
    num_unique_words = len(fdist.items())

    average_word_length = round(sum(len(word) for word in corpus) / len(corpus), 2)
    max_word_length = np.max(np.array([len(word) for word in corpus]))
    average_sentence_length = round(len(corpus) / len(sentences), 2)

    # Hapaxes are words that appear only once
    num_hapaxes = len(fdist.hapaxes())
    top_n_words = fdist.most_common(10)
        
    print("---------------------Article {%d}-----------------------" % article_id)
    print(rubric)
    print('\tMaximum word length in headline:'.ljust(25), max_word_length)
    print('\tAverage length of a word (letterwise):'.ljust(25), average_word_length)
    print('\tAverage length headline length(wordwise):'.ljust(25), average_sentence_length)
    print('\tNum Sentences:'.ljust(25), len(sentences))
    print('\tNum Words:'.ljust(25), num_words)
    print('\tNum Unique Words:'.ljust(25), num_unique_words)
    print('\tNum Hapaxes:'.ljust(25), num_hapaxes)
    print('\tTop 10 Most Frequent Words (stop words):\n\t\t', \
            '\n\t\t'.join(['%s (%s)'
            % (w[0], w[1]) for w in top_n_words]))
    print("-------------------------------------------------------")
    data = []

    for word,freq in fdist.most_common(10):
        data.append({'Word':word,'Frequency':freq})

    df = pd.DataFrame(data)

    # Create a bar graph with the most commonly used descriptors
    df['Word']=df['Word'].str.capitalize()
    
    fig, axes = plt.subplots(nrows=1, ncols=2)
    df.plot(kind='barh', x='Word',y='Frequency', ax=axes[0], figsize=(15,5),color='#cb101c')
    df.plot(kind='line', x='Word',y='Frequency', ax=axes[1], figsize=(15,5))

In [ ]:
def _score_sentences(sentences, important_words):
    N = 3  # Number of words to consider
    CLUSTER_DIST = 3  # Distance between words to consider
    scores = []
    sentence_idx = -1

    for s in [nltk.tokenize.word_tokenize(s) for s in sentences]:

        sentence_idx += 1
        word_idx = []

        # For each word in the word list...
        for w in important_words:
            try:
                # Compute an index for where any important words occur in the sentence.

                word_idx.append(s.index(w[0]))
            except ValueError: # w not in this particular sentence
                pass

        word_idx.sort()

        # It is possible that some sentences may not contain any important words at all.
        if len(word_idx)== 0: continue

        # Using the word index, compute clusters by using a max distance threshold
        # for any two consecutive words.

        clusters = []
        cluster = [word_idx[0]]
        i = 1
        while i < len(word_idx):
            if word_idx[i] - word_idx[i - 1] < CLUSTER_DIST:
                cluster.append(word_idx[i])
            else:
                clusters.append(cluster[:])
                cluster = [word_idx[i]]
            i += 1
        clusters.append(cluster)

        # Score each cluster. The max score for any given cluster is the score 
        # for the sentence.

        max_cluster_score = 0
        for c in clusters:
            significant_words_in_cluster = len(c)
            total_words_in_cluster = c[-1] - c[0] + 1
            score = 1.0 * significant_words_in_cluster \
                * significant_words_in_cluster / total_words_in_cluster

            if score > max_cluster_score:
                max_cluster_score = score

        scores.append((sentence_idx, score))

    return scores

In [ ]:
def pick_top_headlines_by_score(NUM_OF_TOP_WORDS):
    
    top_n_words = fdist.most_common(NUM_OF_TOP_WORDS)
    normalized_sentences = [s.lower() for s in sentences]
    NUM_TOP_SCORED_HEADLINES = 5  # Number of sentences to return for a "top n" summary
    scored_sentences = _score_sentences(normalized_sentences, top_n_words)

    # Summarization Approach 1:
    # Filter out nonsignificant sentences by using the average score plus a
    # fraction of the std dev as a filter

    avg = np.mean([s[1] for s in scored_sentences])
    std = np.std([s[1] for s in scored_sentences])
    mean_scored = [(sent_idx, score) for (sent_idx, score) in scored_sentences
                   if score > avg + 0.5 * std]

    # Summarization Approach 2:
    # Another approach would be to return only the top N ranked sentences

    top_n_scored = sorted(scored_sentences, key=lambda s: s[1])[-NUM_TOP_SCORED_HEADLINES:]
    top_n_scored = sorted(top_n_scored, key=lambda s: s[0])
    
    print("-------------------------------------------------------")
    print("-------HEADLINES(Picked/blue -- original/black)--------")
    top_n_summary=[sentences[idx] for (idx, score) in top_n_scored]
    mean_scored_summary=[sentences[idx] for (idx, score) in mean_scored]
    for sen in sentences:
        if (sen) in set(mean_scored_summary and top_n_summary):
            print("\x1b[94m"+sen+"\x1b[0m"+"\t")
        else:
            print(sen)
    print("-------------------------------------------------------")
    for (idx, score) in mean_scored:
        print(sentences[idx], score)
    print("------------------------------------------------------")
    print("-------------------Lemmatized text--------------------")
    print("------------------------------------------------------")
    for top_sentence in top_n_summary:
        lemmatized_sentence, analysis_info = informative_preprocess_headline(top_sentence)
        print("Original =>\t"+top_sentence)
        print("Preprocessed =>\t"+"\x1b[35m" +' '.join(lemmatized_sentence) +"\x1b[0m")
        for i in analysis_info:
            print("word info:", i['analysis'], i['text'])
        print("=================================================")
    print("-----------------------------------------------------")

In [ ]:
headline_stats() # article_id being passed as parameter, data aggregation is also a possible option for later.
# print the most suitable headlines ranked by score, based on the simple scoring approaches we have used.
pick_top_headlines_by_score(10)

#### Comment: 
From the resul.ts that we have obtained, we can see that the way we scored the sentences is good, but can be improved a bit.. for instance, the sentence with the highest score, has the word "Messanger", which is redundant, also it does not give any information like when this will happen and is it for all phones or only for old, etc. This happens because the score mechanism that we have used has no information about the entities that each word represnts, having information about the type/entities that a word represnts, can help to have a better scoring mechanism if we give entites like "Date" more importance, while giving words like "Messanger" less.

In [ ]:
def get_pos_entity():
        pos_tagged_tokens = [nltk.pos_tag(t) for t in tokens]

    # Flatten the list since we're not using sentence structure
    # and sentences are guaranteed to be separated by a special
    # POS tuple such as ('.', '.')

    pos_tagged_tokens = [token for sent in pos_tagged_tokens for token in sent]

    all_entity_chunks = []
    previous_pos = None
    current_entity_chunk = []
    for (token, pos) in pos_tagged_tokens:

        if pos == previous_pos and pos.startswith('NN'):
            current_entity_chunk.append(token)
        elif pos.startswith('NN'):
            if current_entity_chunk != []:

                # Note that current_entity_chunk could be a duplicate when appended,
                # so frequency analysis again becomes a consideration

                all_entity_chunks.append((' '.join(current_entity_chunk), pos))
            current_entity_chunk = [token]

        previous_pos = pos

    # Store the chunks as an index for the document
    # and account for frequency while we're at it...

#     article_headlines = {}
#     for c in all_entity_chunks:
#         article_headlines['entities'][c] = article_headlines['entities'].get(c, 0) + 1

#     # For example, we could display just the title-cased entities

#     proper_nouns = []
#     for (entity, pos) in article_headlines['entities']:
#         if entity.istitle():
#             print('\t%s (%s)' % (entity, article_headlines['entities'][(entity, pos)]))


In [ ]:
from ipymarkup import show_span_box_markup, show_dep_markup

### Bag Of Words for the headlines of each article

### n-grams model 

### LDA using TF-IDF

#### Intertopic Distance Plot can help you learn about how topics relate to each other, including potential higher-level structure between groups of topics.